# S-matrix modeling of acoustic scattering
This notebook is intended to demonstrate the usefulness of simulations and acoustic modeling with the S-matrix formulation. 
The compact S-matrix offers a highly flexible alternative to full-fledged FDTD simulations along with several advantages over them. 

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import numpy.fft as npft
from logzero import logger

from scipy.special import hermite

import functools as ftls

import Simulator 
import Pulse as pulse
import smatrix as smat

%matplotlib widget

# User edit

In [ ]:
num_grid_points = 10000
stack_thickness = 0.1
num_steps = 50000

# pulse details
HG = 2
pulse_location = -0.3
pulse_width = 0.01

def Square( chi ):
    return ( np.abs( chi ) <= 0.5 ).astype( float )


# Material-free simulation
This is done in order to get the incident wave. 

In [ ]:
stack_materials = [ [ 1, 1 ] ]
stack_layers = [ 'air' ]
interface_locations = list( np.linspace( -stack_thickness/2, stack_thickness/2, len( stack_layers )-1 ) )
interface_locations.insert( 0, -0.5 )
interface_locations.append( 0.5 )
stack = []
for l, m, left, right in zip( stack_layers, stack_materials, interface_locations[:-1], interface_locations[1:] ):
    stack.append( [ l, m[0], m[1], left, right ] )

# defining initial waveform with constant-phase spectrum
x = np.linspace( -0.5, 0.5, num_grid_points )
chi = ( x - pulse_location ) / pulse_width

# initial = Square( chi )
initial = np.exp( -0.5 * chi**2 ) * hermite(HG)( chi )
# initial = pulse.MysteryPulse( x, pulse_location, pulse_width, order=HG )
# initial = np.roll( initial, np.argmin( np.abs( x-pulse_location ) )-x.size//2 )

initial /= np.abs( initial ).max() 
# initial -= initial.mean()

sim = Simulator.Simulator( num_grid_points=num_grid_points, stack=stack )
sim.AddWaveform( initial )
sim.SetUpMonitors( [ -stack_thickness/2, stack_thickness/2 ], visualization=True )
sim.Run( visualize=False, n_iters=num_steps ) #, filename='demo.mp4' )


# Extracting incident wave

In [ ]:
waves = [ val for _, val in sim.monitors.items() ] 
# waves[0] = np.pad( initial, ( ( 0, len( waves[0] ) - len( initial ) ) ) )
t = sim.dt * np.arange( len( waves[0] ) )

with plt.style.context( '~/Templates/custom.mplstyle' ):
    plt.figure( figsize=( 12, 5 ) )
    for n in range( 2 ):
        plt.plot( t, waves[n], label=f'Rx{n}', linewidth=2 )
    plt.grid()
    plt.legend( title='Simulated measurements' )
    plt.title( 'Wave simulation - free space' )
    plt.show()

incident_wave = np.array( waves[0] )

# Stack simulation

In [ ]:
# multi-layer stack
# stack_materials = [ [ 1, 1 ], [ 4., 1. ], [ 6., 1. ], [ 3.75, 1. ], [ 6., 1. ], [ 1, 1 ] ]
# stack_layers = [ 'air', 'block1', 'block2', 'block3', 'block4', 'air' ]

# two-layer stack
stack_materials = [ [ 1, 1 ], [ 6., 1. ], [ 3., 1. ], [ 1, 1 ] ]
stack_layers = [ 'air', 'block1', 'block2', 'air' ]

# # # three-layer stack
# stack_materials = [ [ 1, 1 ], [ 6., 1. ], [ 8., 1. ],  [ 6., 1. ], [ 1, 1 ] ]
# stack_layers = [ 'air', 'block1', 'block2', 'block1', 'air' ]


interface_locations = list( np.linspace( -stack_thickness/2, stack_thickness/2, len( stack_layers )-1 ) )
interface_locations.insert( 0, -0.5 )
interface_locations.append( 0.5 )
stack = []
for l, m, left, right in zip( stack_layers, stack_materials, interface_locations[:-1], interface_locations[1:] ):
    stack.append( [ l, m[0], m[1], left, right ] )

# # defining initial waveform
# initial = -pulse.HermiteGauss( np.linspace( -0.5, 0.5, num_grid_points ), location=-0.3, spread=0.025, order=HG )
# initial /= np.abs( initial ).max() 

sim = Simulator.Simulator( num_grid_points=num_grid_points, stack=stack )
sim.AddWaveform( initial )
plt.figure(); sim.SetUpPlot( figsize=( 12, 4 ) )
sim.SetUpMonitors( [ -stack_thickness/2, stack_thickness/2 ], visualization=True )
sim.Run( visualize=False, n_iters=num_steps ) #, filename='demo.mp4' )


# Extracting simulated waveforms

In [ ]:
waves_measured = [ np.array( val ) for _, val in sim.monitors.items() ] 
wave_reflected = [ a-b for a, b in zip( waves_measured[0], incident_wave ) ]
energy = []
with plt.style.context( '~/Templates/custom.mplstyle' ):
    plt.figure( figsize=( 12, 5 ) )
    for n, ( wav, lbl ) in enumerate( zip( [ incident_wave, wave_reflected, waves_measured[1] ], [ 'Incident', 'Reflected', 'Transmitted' ] ) ):
        E = sim.dt*( np.abs( wav )**2 ).sum()
        energy.append( E )
        logger.info( f'{lbl} energy\t= {E:.3e}' )
        plt.plot( t, wav, label=lbl, linewidth=2 )
    plt.grid()
    plt.legend( title='Simulated pulses' )
    plt.title( 'Time-domain simulation' )
    plt.xlabel( 't (arb. units)' )
    plt.ylabel( 'Wave amp (arb. units)' )

Ert = energy[1]+energy[2]
error = np.abs( Ert-energy[0] )/energy[0]
logger.info( f'Energy conservation error: {error*100.:.7f}%' )



In [ ]:
speed, density = zip( *stack_materials )
Z = np.array( [ a*b for a, b in zip( speed, density ) ] ) # acoustic impedances
R = ( ( Z[0]-Z[1] )/( Z[1]+Z[0] ) )**2 # energy reflection coefficient


# Fourier representation of waves
This is the ground truth against which the output of the S-matrix will be computed , for each frequency.

In [ ]:
freqs = npft.fftfreq( len( incident_wave ), d=sim.dt )
freqs = freqs[ freqs >= 0. ] 
df = ( freqs[1:] - freqs[:-1] ).mean()
freqs = np.array( list( freqs ) + [ freqs[-1] + df ] )

fwav_inc = npft.rfft( incident_wave, norm='ortho' )
fwav_ref = npft.rfft( wave_reflected, norm='ortho' )
fwav_trn = npft.rfft( waves_measured[1], norm='ortho' )

logger.info( f'Fourier components\t= {freqs.size}' )
logger.info( f'Frequency step\t= {df:.3e}' )

In [ ]:
with plt.style.context( '~/Templates/custom.mplstyle' ):
    ax = plt.figure( figsize=( 12, 5 ) ).subplots( 1, 2, sharex=True )
    
    for n, ( wav, lbl ) in enumerate( zip( [ fwav_inc, fwav_ref, fwav_trn ], [ 'Incident', 'Reflected', 'Transmitted' ] ) ):
        ax[0].plot( freqs, np.abs( wav ), label=lbl, linewidth=2 )
    # ax[0].set_xlim( [ 0., 35. ] )
    ax[0].grid()
    ax[0].legend()
    ax[0].set_title( 'Amplitude' )
    ax[0].set_xlabel( 'f (inverse units)' )
    ax[0].set_ylabel( '|FFT|' )
    ax[0].set_xlim( [ 0., 1/pulse_width ] )
    
    # ax.figure( figsize=( 12, 5 ) )
    for n, ( wav, lbl ) in enumerate( zip( [ fwav_inc, fwav_ref, fwav_trn ], [ 'Incident', 'Reflected', 'Transmitted' ] ) ):
        ax[1].plot( freqs, np.unwrap( np.angle( wav ) )[:freqs.size], label=lbl, linewidth=2 )
    # ax[1].set_xlim( [ 0., 3/pulse_width ] )
    ax[1].set_ylim( [ -100., 10. ] )
    ax[1].grid()
    ax[1].legend()
    ax[1].set_title( 'Phase' )
    ax[1].set_xlabel( 'f (inverse units)' )
    ax[1].set_ylabel( '$\\angle$(FFT)' )

    plt.suptitle( 'Wave spectra (\"ground truth\")' )
    plt.tight_layout()
    plt.show()


df = ( freqs[1:] - freqs[:-1] ).mean()
Ef = []
for n, ( wav, lbl ) in enumerate( zip( [ fwav_inc, fwav_ref, fwav_trn ], [ 'Incident', 'Reflected', 'Transmitted' ] ) ):
    power = np.abs( wav )**2
    temp = df*( 0.5*( power[0] + power[-1] ) + power[1:-1].sum() ) / ( 2.*np.pi )
    logger.info( f'{lbl} spectrum total energy = {temp:.7e}' )
    Ef.append( temp )
dE_E = np.abs(Ef[1]+Ef[2]-Ef[0])/Ef[0]
logger.info( f'Energy conservation error = {dE_E*100.:.7e}%' )
logger.info( f'Energy conservation error (in multiples of machine zero) = {dE_E/np.finfo( float ).eps:.15f}' )
logger.info( f'Theoretical reflected energy (first bounce) = {Ef[0]*R:.7e}' )

# Harmonic analysis
Uncomment the next cell if the input pulse is a square pulse. 
Instead of relying on the square pulse spectrum obtained from the simulation (subject to discretization errors), it's better to analytically form the spectrum that will go into the S-matrix. 

In [ ]:
# fwav_inc = np.abs( fwav_inc ).max() * np.sinc( freqs * pulse_width / speed[0] ).astype( complex )
# delay = ( -stack_thickness/2. - pulse_location ) / speed[0]
# phase_ramp = np.exp( -2.j * np.pi * freqs * delay )
# fwav_inc *= phase_ramp

In [ ]:
input = np.concatenate( [ arr[np.newaxis,:] for arr in [ fwav_inc, np.zeros( fwav_inc.shape ) ] ], axis=0 )

# build propagator
if len( stack_layers ) == 3:
    logger.warning( 'Single stack. ' )
    corr = float( speed[1] )
    propagator = ftls.partial( smat.SMatrix, length=corr*stack_thickness, speed=float( speed[1] ), Z=Z )
else: 
    logger.warning( 'Composite stack. ' )
    length = [b-a for b, a in zip( interface_locations[2:-1], interface_locations[1:-2] ) ]
    propagators = [ 
        ftls.partial( smat.SMatrix, length=s*l, speed=s, Z=Z[n:n+3] )
        for n, ( l, s ) in enumerate( zip( length, speed[1:-1] ) )
    ]


# all_outputs = np.empty( input.shape ).astype( complex )
all_outputs = []
start_time = time.time()
for n, omega_test in enumerate( 2.*np.pi*freqs ):
    
    if len( stack_layers ) == 3: 
        S = propagator( omega=omega_test ).S
        output = S @ input[:,n][:,np.newaxis]
    else:
        S = ftls.reduce( lambda x, y: x*y, [ prop( omega=omega_test ) for prop in propagators ] )
        # print( np.abs( np.linalg.eig( np.conj( S.S.T ) @ S.S )[0] ) )
        output = S.S @ input[:,n][:,np.newaxis]

    all_outputs.append( output )
    # all_outputs[:,n] = output.squeeze()

elapsed_time = time.time() - start_time
logger.info( f'S-matrix simulation took {elapsed_time:.6f} seconds to reconstruct {freqs.size} Fourier components. ' )

all_outputs = np.concatenate( all_outputs, axis=1 )

In [ ]:
with plt.style.context( '~/Templates/custom.mplstyle' ):
    ax = plt.figure( figsize=( 12, 5 ) ).subplots( 1, 2, sharex=True )
    
    for wav, lbl in zip( [ fwav_inc, all_outputs[1], all_outputs[0] ], [ 'Incident', 'Reflected', 'Transmitted' ] ):
        ax[0].plot( freqs, np.abs( wav )[:freqs.size], label=lbl, linewidth=2 )
    # ax[0].set_xlim( [ 0., 35. ] )
    ax[0].grid()
    ax[0].legend()
    ax[0].set_title( 'Amplitude' )
    ax[0].set_xlabel( 'f (inverse units)' )
    ax[0].set_ylabel( '|FFT|' )
    ax[0].set_xlim( [ 0., 1/pulse_width ] )
    
    for wav, lbl in zip( [ fwav_inc, all_outputs[1], all_outputs[0] ], [ 'Incident', 'Reflected', 'Transmitted' ] ):
        ax[1].plot( freqs, np.unwrap( np.angle( wav ) )[:freqs.size], label=lbl, linewidth=2 )
    ax[1].set_ylim( [ -100., 10. ] )
    ax[1].grid()
    ax[1].legend()
    ax[1].set_title( 'Phase' )
    ax[1].set_xlabel( 'f (inverse units)' )
    ax[1].set_ylabel( '$\\angle$(FFT)' )

    plt.suptitle( 'Wave spectra from S-matrix propagator' )
    plt.tight_layout()
    
df = ( freqs[1:] - freqs[:-1] ).mean()
Ef = []
for wav, lbl in zip( [ fwav_inc[:-1], all_outputs[1], all_outputs[0] ], [ 'Incident', 'Reflected', 'Transmitted' ] ):
    power = np.abs( wav )**2
    temp = df*( 0.5*( power[0] + power[-1] ) + power[1:-1].sum() ) / ( 2. * np.pi )
    logger.info( f'{lbl} spectrum total energy = {temp:.7e}' )
    Ef.append( temp )
dE_E = np.abs(Ef[1]+Ef[2]-Ef[0])/Ef[0]
logger.info( f'Energy conservation error = {dE_E*100.:.7e}%' )
logger.info( f'Energy conservation error (in multiples of machine zero) = {dE_E/np.finfo( float ).eps:.15f}' )
logger.info( f'Theoretical reflected energy (first bounce) = {Ef[0]*R:.7e}')

plt.show()

# Simulated waveforms from S-matrix
Compare these to the simulated waveforms plotted earlier. 

In [ ]:
back = npft.irfft( fwav_inc, norm='ortho' )
back_r = npft.irfft( all_outputs[1], norm='ortho' ) # negative sign because it's moving in theopposite direction in space
back_t = npft.irfft( all_outputs[0], norm='ortho' )

In [ ]:
with plt.style.context( '~/Templates/custom.mplstyle' ):
    plt.figure( figsize=( 12, 5 ) )
    for n, ( wav, lbl ) in enumerate( zip( [ back, back_r, back_t ], [ 'Incident', 'Reflected', 'Transmitted' ] ) ):
        E = sim.dt*( np.abs( wav )**2 ).sum()
        energy.append( E )
        logger.info( f'{lbl} energy\t= {E:.3e}' )
        plt.plot( t[:wav.size], wav, label=lbl, linewidth=2 )
    plt.grid()
    plt.legend( title='Simulated pulses' )
    plt.title( 'S-matrix simulation' )
    plt.xlabel( 't (arb. units)' )
    plt.ylabel( 'Wave amp (arb. units)' )
    

Ert = energy[1]+energy[2]
error = np.abs( Ert-energy[0] )/energy[0]
logger.info( f'Energy conservation error: {error*100.:.7f}%' )

plt.show()



# Comparison plots

In [ ]:
ax = plt.figure( figsize=( 12, 5 ) ).subplots( 1, 3, sharex=True, sharey=True )

with plt.style.context( '~/Templates/custom.mplstyle' ):
    for n, ( wav1, wav2, lbl ) in enumerate( zip( [ incident_wave, wave_reflected, waves_measured[1] ], [ back, back_r, back_t ], [ 'Incident', 'Reflected', 'Transmitted' ] ) ):
        ax[n].plot( t, wav1, label='Time-domain' )
        ax[n].plot( t, wav2, label='S-matrix' )
        ax[n].grid()
        ax[n].legend()
        ax[n].set_title( lbl )
        logger.info( f'{lbl} error: {np.abs( wav1-wav2 ).sum()/np.array( wav1 ).size:.3e}' )
    plt.suptitle( 'Comparison plots' )
    plt.tight_layout()
